# Running a Machine Learning Model on Quantum Hardware

### Imports and Environment Setup
This cell imports all necessary libraries for data handling (`pandas`, `NumPy`), preprocessing and modelling (`scikit-learn`), plotting (`matplotlib`), and quantum machine learning (`Qiskit` and its machine-learning extensions). It also ensures the `Heatmaps/` directory exists for saving later figures.

In [1]:
##------------ Import Required Python Packages ------------
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

##------------ Import Required Machine Learning Packages ------------
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC 


##------------ Import Required Qiskit Packages ------------
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityStatevectorKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit_ibm_runtime       import QiskitRuntimeService, SamplerV2 as Sampler

##------------ Create/Find OS Directory to Save the Heatmap Results ------------
os.makedirs('../Heatmaps', exist_ok=True)

### Data Loading and Preprocessing
This cell reads the raw dataset (`bots_vs_users.csv`) into a DataFrame, drops features with over 75 % missing values, imputes and flags missing entries, encodes boolean and categorical variables, removes duplicate rows, filters out features with very low variance, and finally imputes and standardises the cleaned feature matrix in preparation for PCA and modelling.


In [2]:
#----------- Load Data into DataFrame -----------
df = pd.read_csv('../Dataset/bots_vs_users.csv')    # Read CSV file into pandas DataFrame

#----------- Drop Very Sparse Columns (>75% missing) -----------
missing_frac = df.isnull().mean().sort_values(ascending=False)    # Compute fraction of missing values per column
high_na      = missing_frac[missing_frac > 0.75].index.tolist()   # Identify columns with >75% NaNs
df           = df.drop(columns=high_na)                           # Remove those sparse columns


#----------- Numeric Imputation & Flags -----------
num_cols = df.select_dtypes(include=['float64','int64']).columns.drop('target')   # Numeric columns excluding target
df[num_cols] = df[num_cols].fillna(0)                                             # Impute NaNs with zero
for c in num_cols:
    df[c + '_was_na'] = (df[c] == 0).astype(int)                                  # Add binary flag for imputed entries


#----------- Boolean Mapping & One-Hot Encoding for Categoricals -----------
bool_cols = [
    c for c in df.columns
    if df[c].dtype == 'object' and set(df[c].dropna().unique()) <= {'True','False'}
]                                                                                 # Detect True/False columns
for c in bool_cols:
    df[c] = df[c].map({'True':1,'False':0})                                       # Map boolean strings to 0/1
cat_cols = df.select_dtypes(include=['object']).columns                           # Remaining categorical columns
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)                        # One-hot encode, drop first level

#----------- Drop Duplicate Rows -----------
df = df.reset_index(drop=True)                       # Reset index after modifications
y_full = df['target']                                # Save target series for later
before = len(df)                                     # Count rows before deduplication
df = df.drop_duplicates()                            # Remove exact duplicate records
print(f"Dropped {before - len(df)} duplicate rows")  # Report number of duplicates removed


#----------- Low-Variance Feature Filter (<1%) -----------
X_full = df.drop(columns=['target'])                              # Separate features
sel    = VarianceThreshold(threshold=0.01)                        # Initialise low-variance selector
X_sel  = sel.fit_transform(X_full)                                # Filter out features with <1% variance
kept   = X_full.columns[sel.get_support()]                        # List of retained feature names
print(f"Kept {len(kept)} features after low-variance filtering")   # Report retained feature count

#----------- Rebuild Cleaned DataFrame & Reattach Target -----------
df_clean = pd.DataFrame(X_sel, columns=kept)                      # Construct DataFrame from filtered features
df_clean['target'] = y_full.loc[df.index].values                  # Reattach the target values

#----------- Final Split into Features/Target, Imputation, Normalization & Standardization -----------
X = df_clean.drop(columns=['target'])                             # Final feature matrix
y = df_clean['target']                                            # Final target vector

# Impute any remaining missing values with the mean
imputer = SimpleImputer(strategy='mean')                          # Mean-value imputer
X_imp   = imputer.fit_transform(X)                                # Impute features

# Normalise features to [0,1]
norm_scaler = MinMaxScaler()                                       # Min–Max normaliser
X_norm      = norm_scaler.fit_transform(X_imp)                     # Scale each feature into [0,1]

# Standardise to zero mean and unit variance
std_scaler = StandardScaler()                                      # Z-score standardiser
X_scaled  = std_scaler.fit_transform(X_norm)                       # Transform normalised data

Dropped 2651 duplicate rows
Kept 74 features after low-variance filtering


### PCA reduction & train/test split

In this section we:

1. Apply **Principal Component Analysis** (PCA) to reduce the pre-scaled feature matrix down to `n_components = 3` dimensions.  
2. **Subsample** the reduced data to 50 examples (while preserving class balance).  
3. Perform an **80/20 train/test split** on that 50-sample subset, producing `X_train`, `X_test`, `y_train` and `y_test` for downstream kernel estimation and classification.


In [ ]:
#----------- PCA reduction & data splitting ------------
n_components = 3                                            # Number of principal components to retain
pca          = PCA(n_components=n_components)               # Initialise PCA transformer
X_red        = pca.fit_transform(X_scaled)                  # Fit PCA on scaled data and reduce dimensionality

# Subsample fewer points to begin with
X_sub, _, y_sub, _ = train_test_split(
    X_red,                                                  # Reduced feature matrix
    y,                                                      # Labels
    train_size=80,                                          # Keep only 50 samples
    stratify=y,                                             # Preserve class balance
    random_state=42                                         # Fixed seed for reproducibility
)

# Now do the 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_sub,                                                  # Subsampled features
    y_sub,                                                  # Subsampled labels
    test_size=0.2,                                          # Allocate 20% for test set
    stratify=y_sub,                                         # Maintain class proportions
    random_state=42                                         # Fixed seed for reproducibility
)

### Feature map definition & hardware sampler setup

Here we prepare everything needed to run on real IBM Quantum hardware:

1. Define a **ZZFeatureMap** with `reps=1` to encode each real feature vector as a parameterised quantum circuit.  
2. Instantiate the **QiskitRuntimeService** (using your stored API token) and pick the **least-busy real device** capable of handling `n_components` qubits.  
3. Wrap that device in a **Sampler** primitive so we can submit parameterised circuits and receive measurement outcomes.


In [4]:
#----------- Feature Map & Hardware Sampler Setup --------------------------------
# 1.) feature map
feature_map = ZZFeatureMap(feature_dimension=n_components, reps=1)    # Create a ZZFeatureMap to encode each n_components-dimensional vector, with one repetition

# 2.) get least-busy real device
service = QiskitRuntimeService()                                      # Initialise QiskitRuntimeService (loads your saved IBMQ token)
backend = service.least_busy(
    simulator=False,                                                  # Exclude simulators; only consider real devices
    operational=True,                                                 # Filter for devices currently operational
    min_num_qubits=n_components                                       # Ensure the device has at least n_components qubits
)

# 3.) sampler primitive on that backend
sampler = Sampler(mode=backend)                                       # Wrap the chosen real device in a Sampler primitive for runtime execution


C:\Users\mattm\AppData\Local\Temp\ipykernel_4916\1054487773.py:6: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService()                                      # Initialise QiskitRuntimeService (loads your saved IBMQ token)


### Fidelity circuit builder & kernel‐matrix function

Two helper functions:

- `fidelity_circuit(x, y)`:  
  - Prepares the state \|ϕ(x)⟩ via the feature map, then “unprepares” \|ϕ(y)⟩ by appending the inverse circuit,  
  - Measures all qubits to estimate the overlap (fidelity) between ∣ϕ(x)⟩ and ∣ϕ(y)⟩.

- `compute_kernel_matrix(X1, X2, shots=128)`:  
  1. Builds all pairwise fidelity circuits between samples in `X1` and `X2`.  
  2. Transpiles & maps them to the chosen hardware.  
  3. Runs them via the `Sampler` primitive.  
  4. Extracts the probability of the all-zero outcome (∣0…0⟩) from each circuit’s counts.  
  5. Returns an |X1|×|X2| NumPy array of these fidelity estimates, forming the quantum kernel matrix.


In [5]:
#----------- Fidelity Circuit & Kernel Computation Functions -----------------------

def fidelity_circuit(x, y):
    qc            = QuantumCircuit(n_components, n_components)         # Create circuit with n_components qubits and classical bits
    # prepare |φ(x)⟩
    param_dict_x  = dict(zip(feature_map.parameters, x))               # Map feature map parameters to the x-vector
    prep_inst     = feature_map.assign_parameters(param_dict_x) \
                          .to_instruction()                            # Build the parameterised feature-map instruction
    qc.append(prep_inst, range(n_components))                          # Append prepare(φ(x)) on all qubits

    # un-prepare ⟨φ(y)| via inverse
    param_dict_y  = dict(zip(feature_map.parameters, y))               # Map feature map parameters to the y-vector
    unprep_inst   = feature_map.assign_parameters(param_dict_y) \
                          .to_instruction().inverse()                  # Build the inverse instruction for φ(y)
    qc.append(unprep_inst, range(n_components))                        # Append unprepare(φ(y)) on all qubits

    qc.measure_all()                                                   # Measure all qubits into classical bits
    return qc                                                          # Return the constructed fidelity circuit

def compute_kernel_matrix(X1, X2, shots=128):
    # 1) Build all pairwise fidelity circuits
    circs     = [fidelity_circuit(x, y) for x in X1 for y in X2]        # List of |X1|×|X2| circuits
    # 2) Transpile & map to the chosen backend
    circs     = transpile(circs, backend=backend, optimization_level=3) # Optimise circuits for hardware
    # 3) Execute on real device via Sampler primitive
    job       = sampler.run(circs, shots=shots)                         # Submit batch job with specified shots
    results   = job.result()                                            # Retrieve PrimitiveResult iterable
    # 4) Extract P(|0…0⟩) from each measurement result
    probs     = [
        r.data.meas.get_counts().get('0'*n_components, 0) / shots       # Probability of the all-zero outcome
        for r in results
    ]
    # 5) Reshape into kernel matrix
    return np.array(probs).reshape(len(X1), len(X2))                    # Return an |X1|×|X2| NumPy array of fidelities


### Kernel evaluation & classical SVM training

Using our hardware-computed Gram matrices:

1. Compute `K_train = compute_kernel_matrix(X_train, X_train)` and fit an **SVM with a precomputed kernel** on the training data.  
2. Compute `K_test = compute_kernel_matrix(X_test, X_train)` and report the **test accuracy** of the precomputed-kernel SVM on the held-out split.


In [6]:
#----------- Kernel Evaluation & Classical SVM Training -----------------------------

# 1.) Train-kernel
K_train = compute_kernel_matrix(X_train, X_train)         # Compute Gram matrix for training data on hardware
svc     = SVC(kernel='precomputed')                       # Initialise SVM using precomputed kernel
svc.fit(K_train, y_train)                                 # Train SVM on the training Gram matrix

# 2.) Test-kernel + accuracy
K_test  = compute_kernel_matrix(X_test, X_train)          # Compute Gram matrix between test and training data
print("Test accuracy:", svc.score(K_test, y_test))        # Evaluate SVM on test Gram matrix and print accuracy


Test accuracy: 0.8125


### Saving the kernel heatmap

Finally, we visualise—and **save**—the training-kernel matrix:

- Generate a Matplotlib heatmap of `K_train`.  
- Save the figure to `../Benchmarks/kernel_train_n{n_components}_hardware.png` (creating the folder if necessary).  
- Close the figure without rendering it inline in the notebook.


In [ ]:
#----------- Plot & Save Training Kernel Heatmap -------------------
plt.figure(figsize=(6,6))                                         # Create a square figure of size 6×6 inches
plt.imshow(K_train, cmap='viridis')                               # Render the training kernel matrix as a heatmap
plt.title("Quantum Kernel Matrix (Train)")                        # Set the plot title
plt.xlabel("Train index")                                         # Label the x-axis
plt.ylabel("Train index")                                         # Label the y-axis
plt.colorbar(label="P(|0…0⟩)")                                    # Add colourbar showing the probability of |0…0⟩

# Save the figure to the Benchmarks folder
out_path = f"../Heatmaps/kernel_train_n{n_components}_hardware.png"      # Build filename including current PCA dimension
plt.savefig(out_path, bbox_inches='tight')                        # Export the figure without extra whitespace
plt.close()                                                       # Close the figure to suppress inline display